In [1]:
# Импортируем генератор датасетов и задачи
from re_rl.dataset_generator import DatasetGenerator
from re_rl.tasks.arithmetic_task import ArithmeticTask
from re_rl.tasks.linear_task import LinearTask

In [2]:
# Старый способ генерации (без системы сложности)
generator = DatasetGenerator()
dataset = generator.generate_dataset(
    task_types=["linear", "quadratic"],
    languages=["ru", "en"],
    detail_levels=[1, 2, 3],
    tasks_per_type=100,
    use_difficulty=False  # Используем старый способ
)
generator.save_dataset(dataset)

Датасет сохранен в datasets/dataset_20250331_004524.json


## Генерация с системой сложности (difficulty)

Новая система позволяет генерировать задачи с разными уровнями сложности от 1 до 10.

Уровень сложности влияет на:
- **ArithmeticTask**: количество операций, размер чисел, использование дробей/процентов
- **LinearTask**: диапазон коэффициентов
- **QuadraticTask**: сложность корней
- **SystemLinearTask**: размер системы (2x2 до 5x5)
- **GraphTask**: количество узлов (4-25)
- **FutoshikiTask**: размер поля (3x3 до 7x7)

In [ ]:
# Генерация датасета с разными уровнями сложности
generator = DatasetGenerator()

# Генерируем задачи с разными difficulty
dataset_with_difficulty = generator.generate_dataset(
    task_types=["arithmetic", "linear", "quadratic"],
    languages=["ru", "en"],
    difficulties=[1, 3, 5, 7, 10],  # 5 уровней сложности
    detail_levels=[2, 3],
    tasks_per_type=10,
    use_difficulty=True
)

print(f"Сгенерировано {len(dataset_with_difficulty)} задач")

# Посмотрим на распределение по сложности
from collections import Counter
difficulty_counts = Counter(task["difficulty"] for task in dataset_with_difficulty)
print("\nРаспределение по сложности:")
for diff, count in sorted(difficulty_counts.items()):
    print(f"  Сложность {diff}: {count} задач")

In [ ]:
# Примеры ArithmeticTask с разными уровнями сложности
print("Примеры ArithmeticTask:")
print("=" * 60)

for difficulty in [1, 5, 10]:
    task = ArithmeticTask(difficulty=difficulty, language="ru")
    result = task.get_result()
    
    print(f"\nСложность {difficulty}:")
    print(f"  Задача: {result['problem']}")
    print(f"  Ответ: {result['final_answer']}")

In [ ]:
# Использование from_difficulty() для создания задач
print("Использование from_difficulty():")
print("=" * 60)

# LinearTask с высокой сложностью
task = LinearTask.from_difficulty(8, language="ru")
result = task.get_result()
print(f"\nLinearTask (difficulty=8):")
print(f"  Задача: {result['problem']}")
print(f"  Ответ: {result['final_answer']}")

# ArithmeticTask с переопределением параметров
task = ArithmeticTask.from_difficulty(
    5, 
    language="ru",
    num_operations=7,  # Переопределяем количество операций
    use_fractions=True
)
result = task.get_result()
print(f"\nArithmeticTask (difficulty=5, num_operations=7):")
print(f"  Задача: {result['problem']}")
print(f"  Ответ: {result['final_answer']}")

In [ ]:
# Сохраняем датасет с системой сложности
generator.save_dataset(dataset_with_difficulty, "dataset_with_difficulty.json")
print("Датасет сохранён!")